In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import Image, ImageFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Function to remove corrupted images
def removeCorruptedImages(path):
    for filename in os.listdir(path):
        try:
            img = Image.open(os.path.join(path,filename))
            img.verify()
        except (IOError, SyntaxError) as e:
            print('Bad file:', filename)
            os.remove(os.path.join(path,filename))

In [3]:
traindir = r'C:\Users\HP\Desktop\New folder\Yoga Data\Train Data'
validdir = r'C:\Users\HP\Desktop\New folder\Yoga Data\Test Data'

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

bx = 16

train_generator = train_datagen.flow_from_directory(traindir, target_size = (300, 300), batch_size = bx, class_mode = 'categorical')
valid_generator = test_datagen.flow_from_directory(validdir, target_size = (300, 300), batch_size = bx, class_mode = 'categorical')

Found 1081 images belonging to 5 classes.
Found 470 images belonging to 5 classes.


In [5]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras.applications import Xception

# Building the model
base_model = Xception(weights ='imagenet',
                      include_top = False,
                      input_shape = (300, 300, 3))

model = Sequential([
    Flatten(input_shape = base_model.output_shape[1:]),
    Dense(128, activation = 'relu'),
    Dropout(0.2),
    Dense(5, activation = 'softmax')
])

model = Model(inputs = base_model.input, outputs = model(base_model.output))

In [6]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate = 0.0001, momentum = 0.9, nesterov = True),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2, from_logits = True),
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'] 

In [7]:
hist = model.fit(train_generator, validation_data = valid_generator, epochs = 15)

Epoch 1/15



C:\Users\HP\anaconda3\lib\site-packages\keras\src\backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(



 5/68 [=>............................] - ETA: 50:50 - loss: 1.7087 - accuracy: 0.1500

C:\Users\HP\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


68/68 [==============================] - 1186s 17s/step - loss: 1.2747 - accuracy: 0.5717 - val_loss: 0.9331 - val_accuracy: 0.8660
Epoch 2/15
68/68 [==============================] - 962s 14s/step - loss: 0.9853 - accuracy: 0.8215 - val_loss: 0.8661 - val_accuracy: 0.9234
Epoch 3/15
68/68 [==============================] - 929s 14s/step - loss: 0.9232 - accuracy: 0.8714 - val_loss: 0.8271 - val_accuracy: 0.9426
Epoch 4/15
68/68 [==============================] - 1004s 15s/step - loss: 0.8647 - accuracy: 0.9223 - val_loss: 0.8093 - val_accuracy: 0.9532
Epoch 5/15
68/68 [==============================] - 899s 13s/step - loss: 0.8470 - accuracy: 0.9362 - val_loss: 0.8067 - val_accuracy: 0.9489
Epoch 6/15
68/68 [==============================] - 1202s 17s/step - loss: 0.8278 - accuracy: 0.9454 - val_loss: 0.8002 - val_accuracy: 0.9511
Epoch 7/15
68/68 [==============================] - 876s 13s/step - loss: 0.8070 - accuracy: 0.9630 - val_loss: 0.7953 - val_accuracy: 0.9574
Epoch 8/15
68/

In [8]:
# Evaluating the model
val_loss, val_accuracy = model.evaluate(valid_generator)
print(f'Validation loss: {val_loss}')
print(f'Validation accuracy: {val_accuracy}')

30/30 [==============================] - 143s 5s/step - loss: 0.7795 - accuracy: 0.9596
Validation loss: 0.7794526219367981
Validation accuracy: 0.9595744609832764


In [9]:
model.save("Yoga_Pose_Classification_Model.h5")

C:\Users\HP\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
